In [1]:
import subprocess
import yaml
from time import sleep
import os
import numpy as np

In [2]:
def ticcmd(*args):
    return subprocess.check_output(['ticcmd'] + list(args))

def get_position():
    status = yaml.load(ticcmd('-s', '--full'))
    position = status['Current position']
    return position

def set_position(target_position):
    position = get_position()
    ticcmd('--resume', '--position', str(target_position))
    while position != target_position:
        position = get_position()
        print(position, end="\r")
        sleep(0.1)
    #sleep(0.5)
    #print(position)
    
def set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0):
    frequency_y2_adjusted = int(np.rint(co_3*target_frequency**3 + co_2*target_frequency**2 + co_1*target_frequency + co_0 - offset))
    set_position(frequency_y2_adjusted)
    
def reset_position(initial_position):
    set_position(initial_position - 200)
    set_position(initial_position)
    
def set_to_zero():
    zero_position = 0
    ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

In [11]:
initial_frequency = 2.811641
initial_position = -3500
frequencies = [initial_frequency, 2.82      , 2.82080537, 2.82161074, 2.82241611, 2.82322148,
       2.82402685, 2.82483221, 2.82563758, 2.82644295, 2.82724832,
       2.82805369, 2.82885906, 2.82966443, 2.8304698 , 2.83127517,
       2.83208054, 2.83288591, 2.83369128, 2.83449664, 2.83530201,
       2.83610738, 2.83691275, 2.83771812, 2.83852349, 2.83932886,
       2.84013423, 2.8409396 , 2.84174497, 2.84255034, 2.8433557 ,
       2.84416107, 2.84496644, 2.84577181, 2.84657718, 2.84738255,
       2.84818792, 2.84899329, 2.84979866, 2.85060403, 2.8514094 ,
       2.85221477, 2.85302013, 2.8538255 , 2.85463087, 2.85543624,
       2.85624161, 2.85704698, 2.85785235, 2.85865772, 2.85946309,
       2.86026846, 2.86107383, 2.86187919, 2.86268456, 2.86348993,
       2.8642953 , 2.86510067, 2.86590604, 2.86671141, 2.86751678,
       2.86832215, 2.86912752, 2.86993289, 2.87073826, 2.87154362,
       2.87234899, 2.87315436, 2.87395973, 2.8747651 , 2.87557047,
       2.87637584, 2.87718121, 2.87798658, 2.87879195, 2.87959732,
       2.88040268, 2.88120805, 2.88201342, 2.88281879, 2.88362416,
       2.88442953, 2.8852349 , 2.88604027, 2.88684564, 2.88765101,
       2.88845638, 2.88926174, 2.89006711, 2.89087248, 2.89167785,
       2.89248322, 2.89328859, 2.89409396, 2.89489933, 2.8957047 ,
       2.89651007, 2.89731544, 2.89812081, 2.89892617, 2.89973154,
       2.90053691, 2.90134228, 2.90214765, 2.90295302, 2.90375839,
       2.90456376, 2.90536913, 2.9061745 , 2.90697987, 2.90778523,
       2.9085906 , 2.90939597, 2.91020134, 2.91100671, 2.91181208,
       2.91261745, 2.91342282, 2.91422819, 2.91503356, 2.91583893,
       2.9166443 , 2.91744966, 2.91825503, 2.9190604 , 2.91986577,
       2.92067114, 2.92147651, 2.92228188, 2.92308725, 2.92389262,
       2.92469799, 2.92550336, 2.92630872, 2.92711409, 2.92791946,
       2.92872483, 2.9295302 , 2.93033557, 2.93114094, 2.93194631,
       2.93275168, 2.93355705, 2.93436242, 2.93516779, 2.93597315,
       2.93677852, 2.93758389, 2.93838926, 2.93919463, 2.94]

co_3 = 69278.68
co_2 = -609102.93
co_1 = 1790504.24
co_0 = - 1762367.85

frequency_y1 = int(np.rint(co_3*initial_frequency**3 + co_2*initial_frequency**2 + co_1*initial_frequency + co_0))
offset = frequency_y1 - initial_position

counts = [0] * len(frequencies)
number_of_runs = 3
integration_time = 5
counter_frequency = 1/integration_time
sleep_time = 6

In [8]:
set_cavity_frequency(offset, 2.87, co_3, co_2, co_1, co_0)

In [10]:
reset_position(initial_position)

In [6]:
counterlogic.set_count_length(300)
counterlogic.set_count_frequency(counter_frequency)
counterlogic.startCount()
smiq.cw_on()

0

In [ ]:
smiq.set_cw(2.87e9)

In [7]:
smiq.cw_on()

0

In [12]:
for i in range(number_of_runs):
    for i in range(len(frequencies)):
        if i == 0:
            target_frequency = frequencies[i]
            smiq.set_cw(frequency=target_frequency*1e9)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]

        if i != 0:
            target_frequency = frequencies[i]
            set_cavity_frequency(offset, target_frequency, co_3, co_2, co_1, co_0)
            smiq.set_cw(frequency=target_frequency*1e9)
            smiq.cw_on()
            sleep(sleep_time)
            counts[i] = counts[i] + counterlogic.countdata[0,-1]
            
    reset_position(initial_position)
            
print(counts)
print(frequencies)

[8306010.800000001, 8296079.800000001, 8284542.400000001, 8305180.600000001, 8321384.200000001, 8281463.4, 8317885.600000001, 8321447.000000001, 8308257.800000001, 8318558.000000002, 8281627.4, 8289436.200000001, 8323745.4, 8299670.4, 8317346.4, 8336941.800000001, 8341329.4, 8305727.000000001, 8301250.4, 8331216.6, 8285938.4, 8290181.199999999, 8275204.4, 8295976.4, 8301870.800000001, 8295213.0, 8317388.4, 8300969.4, 8283035.4, 8272453.4, 8280177.800000001, 8275736.199999999, 8285413.0, 8253224.4, 8277549.600000001, 8279790.8, 8273873.600000001, 8280894.6, 8249496.4, 8267229.4, 8289775.200000001, 8263894.000000001, 8257889.6, 8250294.0, 8226222.4, 8227222.600000001, 8202901.4, 8246466.6, 8228710.800000001, 8158600.200000001, 8192110.800000001, 8153214.200000001, 8138856.000000001, 8138523.200000001, 8127515.800000001, 8133187.200000001, 8129525.200000001, 8107511.000000001, 8081802.6, 8074366.2, 8087489.2, 8079648.800000001, 8097466.4, 8118121.800000001, 8135723.4, 8137717.200000001, 8

In [ ]:
#Sets current position to 0
zero_position = 0
ticcmd('--exit-safe-start', '--halt-and-set-position', str(zero_position))

#Get current count parameters
counterlogic.get_count_length()
counterlogic.get_count_frequency()

# Brute force way to control anything, pull gui levers.
counter._mw.start_counter_Action.trigger()

In [13]:
get_position()

3439

In [16]:
set_position(-3500)

In [14]:
set_to_zero()

In [ ]:
reset_position(-4800)

In [12]:
smiq.off()


0

In [9]:
np.linspace(2.82, 2.94, 150)

array([2.82      , 2.82080537, 2.82161074, 2.82241611, 2.82322148,
       2.82402685, 2.82483221, 2.82563758, 2.82644295, 2.82724832,
       2.82805369, 2.82885906, 2.82966443, 2.8304698 , 2.83127517,
       2.83208054, 2.83288591, 2.83369128, 2.83449664, 2.83530201,
       2.83610738, 2.83691275, 2.83771812, 2.83852349, 2.83932886,
       2.84013423, 2.8409396 , 2.84174497, 2.84255034, 2.8433557 ,
       2.84416107, 2.84496644, 2.84577181, 2.84657718, 2.84738255,
       2.84818792, 2.84899329, 2.84979866, 2.85060403, 2.8514094 ,
       2.85221477, 2.85302013, 2.8538255 , 2.85463087, 2.85543624,
       2.85624161, 2.85704698, 2.85785235, 2.85865772, 2.85946309,
       2.86026846, 2.86107383, 2.86187919, 2.86268456, 2.86348993,
       2.8642953 , 2.86510067, 2.86590604, 2.86671141, 2.86751678,
       2.86832215, 2.86912752, 2.86993289, 2.87073826, 2.87154362,
       2.87234899, 2.87315436, 2.87395973, 2.8747651 , 2.87557047,
       2.87637584, 2.87718121, 2.87798658, 2.87879195, 2.87959

In [9]:
counterlogic.countdata[0,-1]

922329.0